In [1]:
import os
import shutil
import random

In [2]:
def merge_folders(source_folders, target_folder):
    os.makedirs(target_folder, exist_ok=True)

    for folder in source_folders:
        for file_name in os.listdir(folder):
            source_path = os.path.join(folder, file_name)
            target_path = os.path.join(target_folder, file_name)

            # Copiar solo si es archivo
            if os.path.isfile(source_path):
                shutil.copy(source_path, target_path)
    print(f"Carpetas {source_folders} combinadas en {target_folder}")

In [3]:
# Carpetas
fall_imgs = 'dataset/Fall'
nofall_imgs = 'dataset/Non_Fall'
fall_labels = 'dataset/Fall_label'
nofall_labels = 'dataset/Non_Fall_label'

# Carpeta destino
merged_imgs = 'dataset/all_imgs'
merged_labels = 'dataset/all_labels'

# Unir
merge_folders([fall_imgs, nofall_imgs], merged_imgs)
merge_folders([fall_labels, nofall_labels], merged_labels)

Carpetas ['dataset/Fall', 'dataset/Non_Fall'] combinadas en dataset/all_imgs
Carpetas ['dataset/Fall_label', 'dataset/Non_Fall_label'] combinadas en dataset/all_labels


In [4]:
txt_path = "to_delete.txt"   # <- tu lista
# 1. Leer los nombres que quieres borrar
with open(txt_path, "r") as f:
    nombres_a_borrar = [linea.strip() for linea in f if linea.strip()]

# 2. Borrar cada imagen y su label
for nombre in nombres_a_borrar:
    # Asumimos que las imágenes son .jpg y los labels son .txt (ajusta si es necesario)
    posibles_archivos = [
        os.path.join(merged_imgs, f"{nombre}.jpg"),
        os.path.join(merged_imgs, f"{nombre}.png"),
        os.path.join(merged_imgs, f"{nombre}.jpeg"),
        os.path.join(merged_labels, f"{nombre}.txt"),
    ]

    for archivo in posibles_archivos:
        if os.path.exists(archivo):
            os.remove(archivo)
            print(f"Eliminado: {archivo}")
        else:
            print(f"No encontrado (ok): {archivo}")

Eliminado: dataset/all_imgs\02_3_392.jpg
No encontrado (ok): dataset/all_imgs\02_3_392.png
No encontrado (ok): dataset/all_imgs\02_3_392.jpeg
No encontrado (ok): dataset/all_labels\02_3_392.txt
Eliminado: dataset/all_imgs\02_3_397.jpg
No encontrado (ok): dataset/all_imgs\02_3_397.png
No encontrado (ok): dataset/all_imgs\02_3_397.jpeg
No encontrado (ok): dataset/all_labels\02_3_397.txt
Eliminado: dataset/all_imgs\03_3_622.jpg
No encontrado (ok): dataset/all_imgs\03_3_622.png
No encontrado (ok): dataset/all_imgs\03_3_622.jpeg
No encontrado (ok): dataset/all_labels\03_3_622.txt
Eliminado: dataset/all_imgs\03_3_623.jpg
No encontrado (ok): dataset/all_imgs\03_3_623.png
No encontrado (ok): dataset/all_imgs\03_3_623.jpeg
No encontrado (ok): dataset/all_labels\03_3_623.txt
Eliminado: dataset/all_imgs\03_3_625.jpg
No encontrado (ok): dataset/all_imgs\03_3_625.png
No encontrado (ok): dataset/all_imgs\03_3_625.jpeg
No encontrado (ok): dataset/all_labels\03_3_625.txt
Eliminado: dataset/all_imgs\03

In [12]:
def split_dataset(images_dir, labels_dir, output_dir, train_ratio=0.8, val_ratio=0.12, test_ratio=0.08, seed=42):
    # Crear carpetas de salida
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_dir, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_dir, split, 'labels'), exist_ok=True)

    fall_files = []
    nofall_files = []

    # Recorrer imágenes y clasificar según el label
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    for img_file in image_files:
        label_file = img_file.rsplit('.', 1)[0] + '.txt'
        label_path = os.path.join(labels_dir, label_file)

        if not os.path.exists(label_path):
            continue  # Saltar si no tiene label

        # Leer el label para ver si es "fall" o "no fall"
        with open(label_path, 'r') as f:
            contenido = f.read().strip()

            if not contenido:
                continue  # Saltar si el label está vacío

            clases = [int(line.split()[0]) for line in contenido.splitlines()]

            if 1 in clases:
                fall_files.append(img_file)
            else:
                nofall_files.append(img_file)

    print(f"Total Fall: {len(fall_files)}, Total No Fall: {len(nofall_files)}")

    # Mezclar aleatoriamente cada grupo
    random.seed(seed)
    random.shuffle(fall_files)
    random.shuffle(nofall_files)

    def dividir_lista(lista):
        total = len(lista)
        train_size = int(total * train_ratio)
        val_size = int(total * val_ratio)

        train_files = lista[:train_size]
        val_files = lista[train_size:train_size + val_size]
        test_files = lista[train_size + val_size:]

        return train_files, val_files, test_files

    # Dividir cada clase
    fall_train, fall_val, fall_test = dividir_lista(fall_files)
    nofall_train, nofall_val, nofall_test = dividir_lista(nofall_files)

    # Unir las listas
    train_files = fall_train + nofall_train
    val_files = fall_val + nofall_val
    test_files = fall_test + nofall_test

    # Volver a mezclar dentro de cada split
    random.shuffle(train_files)
    random.shuffle(val_files)
    random.shuffle(test_files)

    splits = [('train', train_files), ('val', val_files), ('test', test_files)]

    # Copiar archivos
    for split_name, split_files in splits:
        for img_file in split_files:
            label_file = img_file.rsplit('.', 1)[0] + '.txt'

            shutil.copy(os.path.join(images_dir, img_file), os.path.join(output_dir, split_name, 'images', img_file))
            shutil.copy(os.path.join(labels_dir, label_file), os.path.join(output_dir, split_name, 'labels', label_file))

    print("Separación balanceada completada exitosamente.")

In [13]:
images_dir = "dataset/Multiple_Cameras/all_imgs"
labels_dir = "dataset/Multiple_Cameras/all_labels"
output_dir = "dataset/Multiple_Cameras/cleaned_dataset"

split_dataset(images_dir, labels_dir, output_dir)

Total Fall: 4688, Total No Fall: 5211
Separación balanceada completada exitosamente.
